# **Armonización de las bases de datos**

Antes de empezar a testear los modelos para las bases de datos, vamos a ralizar una armonización entre los datasets, estudiar compatibilidades entre ellos para dejat un solo dataset bien hecho para poder entrenar al modelo.

Claro está, puede hacerse el entrenamiento dataset por dataset, es más, lo haremos. Pero ahora me interesa seguir minando en los datos para dejar un dataset final para el modelo.

Recordemos que lo que seleccionamos para los datasets fueron: 

### **Dataset: KOI_All_Filtrated.csv (9564×15)**

*Columnas y sus descripciones oficiales:*

1. koi_period	Periodo orbital del planeta candidato en días.

2. koi_duration	Duración del tránsito en horas (tiempo desde el inicio hasta el final del tránsito).

3. koi_prad	Radio planetario en radios terrestres.

4. koi_teq	Temperatura de equilibrio (K). Aproximación de la temperatura del planeta suponiendo equilibrio radiativo, albedo de 0.3 y distribución uniforme del calor. 

5. koi_insol	Flujo de insolación recibido por el planeta en unidades del flujo terrestre. 

6. koi_srad	Radio estelar en radios solares.

7. koi_steff	Temperatura efectiva estelar (K). Temperatura fotosférica del anfitrión. 

8. koi_depth	Profundidad del tránsito (ppm). Caída relativa del flujo estelar durante el tránsito.

9. koi_model_snr	Relación señal-ruido del tránsito: profundidad del tránsito normalizada por la incertidumbre media del flujo. 

10. koi_score	Fracción de iteraciones del Robovetter que produjeron una disposición de “CANDIDATE” (métrica probabilística de validación).

11. koi_fpflag_nt	Flag “Not Transit-Like”: indica que la curva no es consistente con un tránsito planetario (artefactos, variables, ruido). 

12. koi_fpflag_ss	Flag “Stellar Eclipse”: indica que el evento es probablemente un eclipse estelar (binaria eclipsante).

13. koi_fpflag_co	Flag “Centroid Offset”: la fuente del tránsito parece provenir de una estrella vecina.

14. koi_fpflag_ec	Flag “Ephemeris Match”: el evento comparte período y época con otra fuente, sugiriendo contaminación o crosstalk.

15. koi_disposition	Clasificación final: CONFIRMED / CANDIDATE / FALSE POSITIVE. 


### **Dataset: K2_All_Filtrated.csv (4004×13)**

*Columnas y sus descripciones oficiales*

1. pl_orbper	Periodo orbital (días). Tiempo que tarda el planeta en completar una órbita.

2. pl_trandur	Duración del tránsito (horas).

3. pl_trandep	Profundidad del tránsito.

4. pl_rade	Radio planetario en radios terrestres.

5. pl_radj	Radio planetario en radios de Júpiter.

6. st_rad	Radio estelar en radios solares. 

7. st_teff	Temperatura efectiva estelar (K).

8. st_mass	Masa estelar en masas solares.

9. st_logg	Gravedad superficial.

10. st_met	Metalicidad estelar en dex.

11. sy_pnum	Número de planetas conocidos en el sistema.

12. sy_snum	Número de estrellas en el sistema.

13. disposition	Clasificación: CONFIRMED / CANDIDATE / FALSE POSITIVE / REFUTED.


### **Dataset: TOI_All_Filtrated.csv (7699×12)**

*Columnas y sus descripciones oficiales*

1. pl_orbper	Periodo orbital (días). Tiempo de una órbita completa.

2. pl_trandurh	Duración del tránsito (horas). Tiempo total del tránsito.

3. pl_trandep	Profundidad del tránsito (ppm). Tamaño de la disminución de flujo durante el tránsito.

4. pl_rade	Radio planetario.

5. pl_insol	Insolación planetaria (flujo recibido, en unidades de flujo terrestre).

6. pl_eqt	Temperatura de equilibrio del planeta (K).

7. st_teff	Temperatura efectiva estelar (K).

8. st_rad	Radio estelar.

9. st_logg	Gravedad superficial estelar.

10. st_tmag	Magnitud TESS (brillo aparente en banda T).

11. st_dist	Distancia estelar (pc).

12. tfopwg_disp	Clasificación TESS (APC=Ambiguous PC, CP=Confirmed Planet, FA=False Alarm, FP=False Positive, KP=Known Planet, PC=Planetary Candidate).

### **ARMONIZACIÓN DE LOS DATASETS NASA (Kepler, K2 y TESS)**

Objetivo: Unificar los tres catálogos (KOI, K2, TOI) en un único dataset global de entrenamiento ML, donde cada fila representa un objeto candidato o confirmado (una señal de tránsito), y cada columna describe variables homogéneas físicas, observacionales o de clasificación.

Decidimos, que seleccionaremos las columnas en común de los 3 datasets de los KOI_All_Filtrated.csv, TOI_All_Filtrated.csv y K2_All_Filtrated.csv.

Dichas columnas que servirán como labels/features se mencionan a continuación:

| **Variable Global**         | **KOI**        | **K2**       | **TOI**       | **Justificación / Reflexión Teórica**                                                                                                                                                         |
| --------------------------- | -------------- | ------------ | ------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `orbital_period_days`       | `koi_period`   | `pl_orbper`  | `pl_orbper`   | Representa el tiempo entre tránsitos consecutivos. Es fundamental para derivar el radio orbital y temperatura de equilibrio. En todos los datasets está en días y es directamente comparable. |
| `transit_duration_hours`    | `koi_duration` | `pl_trandur` | `pl_trandurh` | Mide la duración total del tránsito, en horas, lo cual depende del tamaño de la estrella, del planeta y de la inclinación orbital. Perfectamente equivalente.                                 |
| `transit_depth_ppm`         | `koi_depth`    | `pl_trandep` | `pl_trandep`  | Es la fracción de luz estelar bloqueada por el planeta (en partes por millón). Directamente comparable, depende de (Rₚ/R*)².                                                                  |
| `planet_radius_earth`       | `koi_prad`     | `pl_rade`    | `pl_rade`     | Todos expresan el radio planetario en radios terrestres (R⊕), aunque K2 también da `pl_radj` en R_J. Se unifica en R⊕ (puede convertirse R_J → R⊕ multiplicando por 11.21).                   |
| `equilibrium_temperature_K` | `koi_teq`      | `pl_eqt`     | `pl_eqt`      | Temperatura de equilibrio teórica en Kelvin. Se calcula a partir de la insolación y propiedades estelares, por lo que es análoga.                                                             |
| `insolation_flux_Earth`     | `koi_insol`    | `pl_insol`   | `pl_insol`    | Flujo recibido por el planeta relativo al terrestre. Variable clave para zona habitable. Equivalente conceptual y en unidades.                                                                |
| `stellar_radius_solar`      | `koi_srad`     | `st_rad`     | `st_rad`      | Radio de la estrella en radios solares (R☉). Directamente comparable.                                                                                                                         |
| `stellar_temperature_K`     | `koi_steff`    | `st_teff`    | `st_teff`     | Temperatura efectiva de la estrella. Misma magnitud física y unidades.                                                                                                                        |


### **Armonización de la variable objetivo (target): final_disposition**


| **Valor Global**                   | **KOI (`koi_disposition`)**    | **K2 (`disposition`)**      | **TOI (`tfopwg_disp`)** | **Significado físico / Justificación teórica**                                                                                                                                                                                       |
| ---------------------------------- | ------------------------------ | --------------------------- | ----------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| `CONFIRMED`                        | `CONFIRMED`                    | `CONFIRMED`                 | `CP`, `KP`              | Objetos cuya existencia planetaria fue **confirmada** mediante observaciones o validaciones adicionales (modelos de tránsito, espectroscopía, o coincidencia con exoplanetas previamente conocidos). Representan exoplanetas reales. |
| `CANDIDATE`                        | `CANDIDATE`                    | `CANDIDATE`                 | `PC`, `APC`             | Señales de tránsito que son **probables exoplanetas**, pero aún no validadas con certeza. Suelen requerir observaciones de seguimiento o validación estadística.                                                                     |
| `FALSE_POSITIVE`                   | `FALSE POSITIVE`               | `FALSE POSITIVE`, `REFUTED` | `FP`, `FA`              | Señales que **imitan un tránsito planetario** pero son producidas por otros fenómenos astrofísicos (binarias eclipsantes, ruido instrumental, contaminación de fondo). No representan exoplanetas reales.                            |
| *(Sin clasificación / descartado)* | `NOT DISPOSITIONED`, `UNKNOWN` | *(No aplica)*               | *(No aplica)*           | Registros sin clasificación concluyente o con datos insuficientes. Estos casos se excluyen del dataset de entrenamiento.                                                                                                             |


In [1]:
# Let's cook some code! - Ver el harmonize_intersection.py